In [ ]:
%pip install pmdarima;

     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 16.6MB/s 
     |████████████████████████████████| 8.7MB 28.6MB/s 
  Found existing installation: Cython 0.29.20
    Uninstalling Cython-0.29.20:


In [ ]:
#Check Version
import statsmodels._version
if statsmodels._version.get_versions()['version'] != '0.11.1':
  !pip install --upgrade statsmodels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%cd "/content/drive/My Drive/Colab Notebooks/Time_Series_Projects/Data"

df1 = pd.read_csv('airline_passengers.csv',index_col='Month',parse_dates=True)
df1.index.freq = 'MS' # Month Start

df2 = pd.read_csv('DailyTotalFemaleBirths.csv',index_col='Date',parse_dates=True)
df2.index.freq = 'D' # Month Start
df2 = df2[:120]

df3 = pd.read_csv('TradeInventories.csv',index_col='Date',parse_dates=True)
df3.index.freq = 'MS'

In [ ]:
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

#**ARMA**

In [ ]:
df2['Births'].plot()

### Function for running the augmented Dickey-Fuller test
Since we'll use it frequently in the upcoming forecasts, let's define a function we can copy into future notebooks for running the augmented Dickey-Fuller test. Remember that we'll still have to import <tt>adfuller</tt> at the top of our notebook.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    print('\nResult:')
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(df2['Births'])

In [ ]:
plot_acf(df2['Births'])
plot_pacf(df2['Births']);

In [ ]:
from pmdarima import auto_arima

In [ ]:
#help(auto_arima)

In [ ]:
print(auto_arima(df2['Births'],seasonal=False,start_p=2,start_q=2,trace=True).summary())

In [ ]:
#train test split
train = df2.iloc[:90]
test = df2.iloc[90:]

In [ ]:
model = ARMA(train['Births'],order = (2,2)).fit()

In [ ]:
print(model.summary())

In [ ]:
start = len(train)
end = len(train) + len(test) - 1

In [ ]:
prediction2 = model.predict(start,end).rename('ARMA(2,2) Predictions')

In [ ]:
test.plot(legend = True,figsize=(12,8))
prediction2.plot(legend = True)


In [ ]:
stepwise_fit = auto_arima(df2['Births'],start_p=0,start_q=0,max_p=6,max_q=3,seasonal=False,trace=True)#trace will show the grid search

In [ ]:
print(stepwise_fit.summary())

#ARIMA

In [1]:
df3.plot()

NameError: ignored

In [73]:
stepwise_fit = auto_arima(df1['Thousands of Passengers'],start_p=0,start_q=1,max_p=4,max_q=4,seasonal=True,trace=True,m=12)

Performing stepwise search to minimize aic
Fit ARIMA(0,1,1)x(1,1,1,12) [intercept=True]; AIC=1022.207, BIC=1036.583, Time=1.062 seconds
Fit ARIMA(0,1,0)x(0,1,0,12) [intercept=True]; AIC=1033.479, BIC=1039.229, Time=0.023 seconds
Fit ARIMA(1,1,0)x(1,1,0,12) [intercept=True]; AIC=1022.316, BIC=1033.817, Time=0.443 seconds
Fit ARIMA(0,1,1)x(0,1,1,12) [intercept=True]; AIC=1022.904, BIC=1034.405, Time=0.522 seconds
Fit ARIMA(0,1,0)x(0,1,0,12) [intercept=False]; AIC=1031.508, BIC=1034.383, Time=0.020 seconds
Fit ARIMA(0,1,1)x(1,1,0,12) [intercept=True]; AIC=1022.314, BIC=1033.815, Time=0.321 seconds
Fit ARIMA(0,1,1)x(2,1,1,12) [intercept=True]; AIC=1015.841, BIC=1033.092, Time=4.036 seconds
Near non-invertible roots for order (0, 1, 1)(2, 1, 1, 12); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.998)
Fit ARIMA(0,1,1)x(1,1,2,12) [intercept=True]; AIC=1014.930, BIC=1032.181, Time=3.553 seconds
Fit ARIMA(0,1,1)x(0,1,2,12) [intercept=True]; AIC=102

In [74]:
print(stepwise_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                  144
Model:             SARIMAX(1, 1, 2)x(1, 1, 2, 12)   Log Likelihood                -499.166
Date:                            Sun, 28 Jun 2020   AIC                           1014.331
Time:                                    23:35:56   BIC                           1037.333
Sample:                                         0   HQIC                          1023.678
                                            - 144                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0011      0.030      0.037      0.971      -0.057       0.059
ar.L1         -0.9165      0.078   